**Missing Values**

In [125]:
# load dataset
import pandas as pd
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')
print(X_full.shape)
print(X_test_full.shape)

(1460, 80)
(1459, 79)


In [115]:
# show na values
X_full.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [126]:
# Remove rows with missing target (Price)
X_full.dropna(axis=0, subset=['SalePrice'])
# define target
y = X_full.SalePrice
# exclude target from feature matrix
X_full.drop(["SalePrice"], axis=1)
print(X_full.shape)

(1460, 80)


In [127]:
# include only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])
X.shape

(1460, 37)

In [128]:
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
print(X_train.shape)
print(X_valid.shape)

(1168, 37)
(292, 37)


In [119]:
# show first rows (train)
X_train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
618,619,20,90.0,11694,9,5,2007,2007,452.0,48,...,0,108,0,0,260,0,0,7,2007,314813
870,871,20,60.0,6600,5,5,1962,1962,0.0,0,...,0,0,0,0,0,0,0,8,2009,109500
92,93,30,80.0,13360,5,7,1921,2006,0.0,713,...,0,0,44,0,0,0,0,8,2009,163500
817,818,20,NaN,13265,8,5,2002,2002,148.0,1218,...,150,59,0,0,0,0,0,7,2008,271000
302,303,20,118.0,13704,7,5,2001,2002,150.0,0,...,468,81,0,0,0,0,0,1,2006,205000


In [129]:
# Show missing values
missings = X_train.isna().sum()
missings.loc[lambda x: x > 0].keys().values

array(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype=object)

In [130]:
# define function to calculate MAE
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_ds(X_train, X_valid, y_train, y_valid):
  model = RandomForestRegressor(n_estimators=100, random_state=0)
  model.fit(X_train, y_train)
  preds = model.predict(X_valid)
  return mean_absolute_error(y_valid, preds)

In [131]:
# Strategy for na: Drop columns in training and validation data
reduced_X_train = X_train.drop(missings.loc[lambda x: x > 0].keys().values, axis=1)
reduced_X_valid = X_valid.drop(missings.loc[lambda x: x > 0].keys().values, axis=1)
print(reduced_X_train.shape)

(1168, 34)


In [132]:
# strategy 2: IMPUTATION
from sklearn.impute import SimpleImputer

simpleImputer = SimpleImputer()

imputed_X_train = pd.DataFrame(simpleImputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(simpleImputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

len(imputed_X_train.columns.to_list())


37

In [ ]:
# generate test predictions
